In [1]:
import os
import sys
from src.exception import CustomException
from src.logger import logging
import pandas as pd
import numpy as np

from dataclasses import dataclass

from src.components.data_transformation import DataTransformation
from src.components.data_transformation import DataTransformationConfig

from src.components.model_trainer import ModelTrainerConfig
from src.components.model_trainer import ModelTrainer

from src.components.data_ingestion import DataIngestionConfig
from src.components.data_ingestion import DataIngestion

from src.utils import save_object,evaluate_models


In [2]:
obj=DataIngestion()
train_data,test_data=obj.initiate_data_ingestion()

In [3]:
data_transformation=DataTransformation(freq='D', method='linear')
train_arr,test_arr,_=data_transformation.initiate_data_transformation(train_data,test_data)


/Users/olasuaifan/Desktop/University/projects/Quant_Research/src/components/data_transformation.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[self.date_col] = pd.to_datetime(df[self.date_col])
/Users/olasuaifan/Desktop/University/projects/Quant_Research/src/components/data_transformation.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[self.date_col] = pd.to_datetime(df[self.date_col])
/Users/olasuaifan/opt/anaconda3/envs/quant/lib/python3.11/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.

In [4]:
train_arr

Dates
2020-10-31    10.100000
2020-11-01    10.106667
2020-11-02    10.113333
2020-11-03    10.120000
2020-11-04    10.126667
                ...    
2023-12-27    12.722581
2023-12-28    12.741935
2023-12-29    12.761290
2023-12-30    12.780645
2023-12-31    12.800000
Freq: D, Name: Prices, Length: 1157, dtype: float64

In [5]:
import sys
from dataclasses import dataclass

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import r2_score
import pandas as pd

from src.exception import CustomException
from src.logger import logging

import os
import sys
from dataclasses import dataclass

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import r2_score
import pandas as pd

from src.exception import CustomException
from src.logger import logging

from src.utils import save_object,evaluate_models


In [6]:
models = {
    "Exponential Smoothing": ExponentialSmoothing,
    "SARIMAX": SARIMAX
}

params = {
    "Exponential Smoothing": {
        'trend': ['add', 'mul'],
        'damped_trend': [True, False],
        'seasonal': ['add', 'mul', None],
        'seasonal_periods': [365],
        'initialization_method': ["legacy-heuristic", 'estimated', 'heuristic', None],
        'use_boxcox': [False, True]
    },
    "SARIMAX": {
        'order': (2, 0, 2),
        'seasonal_order': (0, 1, 1, 12)
    }
}

In [7]:
model_report:dict=evaluate_models(train_arr,test_arr,
                                             models=models,param=params)

/Users/olasuaifan/opt/anaconda3/envs/quant/lib/python3.11/site-packages/statsmodels/tsa/holtwinters/model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/olasuaifan/opt/anaconda3/envs/quant/lib/python3.11/site-packages/statsmodels/tsa/holtwinters/model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/olasuaifan/opt/anaconda3/envs/quant/lib/python3.11/site-packages/statsmodels/tsa/holtwinters/model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/olasuaifan/opt/anaconda3/envs/quant/lib/python3.11/site-packages/statsmodels/tsa/holtwinters/model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/olasuaifan/opt/anaconda3/envs/quant/lib/python3.11/site-packages/statsmodels/tsa/holtwinters/model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  

In [8]:
model_report

{'Exponential Smoothing_1': {'class': statsmodels.tsa.holtwinters.model.ExponentialSmoothing,
  'mape': 0.014662456216409,
  'params': {'trend': 'add',
   'damped_trend': True,
   'seasonal': 'add',
   'seasonal_periods': 365,
   'initialization_method': 'legacy-heuristic',
   'use_boxcox': False},
  'r2': 0.8024542574599616},
 'Exponential Smoothing_2': {'class': statsmodels.tsa.holtwinters.model.ExponentialSmoothing,
  'mape': 0.03724401634095469,
  'params': {'trend': 'add',
   'damped_trend': True,
   'seasonal': 'add',
   'seasonal_periods': 365,
   'initialization_method': 'legacy-heuristic',
   'use_boxcox': True},
  'r2': 0.02460649363661427},
 'Exponential Smoothing_3': {'class': statsmodels.tsa.holtwinters.model.ExponentialSmoothing,
  'mape': 0.05430796879811058,
  'params': {'trend': 'add',
   'damped_trend': True,
   'seasonal': 'add',
   'seasonal_periods': 365,
   'initialization_method': 'estimated',
   'use_boxcox': False},
  'r2': -0.8327980953683389},
 'Exponential S

In [9]:
best_model_name, best_model_info = min(
    model_report.items(), key=lambda x: x[1]['mape']
)

best_model_score = best_model_info['mape']
best_model_class = best_model_info['class']
best_model_params = best_model_info['params']

In [10]:
best_model_info

{'class': statsmodels.tsa.holtwinters.model.ExponentialSmoothing,
 'mape': 0.0079300348713607,
 'params': {'trend': 'add',
  'damped_trend': True,
  'seasonal': 'mul',
  'seasonal_periods': 365,
  'initialization_method': 'legacy-heuristic',
  'use_boxcox': False},
 'r2': 0.935679622644676}

In [11]:
mape_subreport = {name: info['mape'] for name, info in model_report.items()}
r2_subreport = {name: info['r2'] for name, info in model_report.items()}


In [12]:
best_r2_score=max(r2_subreport.values())
best_model_name = max(r2_subreport, key=r2_subreport.get)
best_model_info = model_report[best_model_name]

In [13]:
best_model_info

{'class': statsmodels.tsa.holtwinters.model.ExponentialSmoothing,
 'mape': 0.0079300348713607,
 'params': {'trend': 'add',
  'damped_trend': True,
  'seasonal': 'mul',
  'seasonal_periods': 365,
  'initialization_method': 'legacy-heuristic',
  'use_boxcox': False},
 'r2': 0.935679622644676}

In [14]:
import datetime

date_input="2030-12-12"

In [15]:
from src.components.data_transformation import DataTransformation
from src.components.data_transformation import DataTransformationConfig
data_transformation = DataTransformation(freq='D', method='linear')
past_data = data_transformation.general_data_transformation('artifacts')
past_data.reset_index(inplace=True, drop=False)

/Users/olasuaifan/Desktop/University/projects/Quant_Research/src/components/data_transformation.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[self.date_col] = pd.to_datetime(df[self.date_col])


In [16]:
date_dt = pd.to_datetime(date_input)
last_date_in_past_data = past_data['Dates'].max()

In [17]:
from src.pipeline.predict_pipeline import CustomData, PredictPipeline

custom_data = CustomData(Date=datetime.datetime.now(), Prices=0)
days = (date_dt - last_date_in_past_data).days
start_date = last_date_in_past_data + pd.Timedelta(days=1)
future_data = custom_data.get_future_data_as_data_frame(Days=days, start_date=start_date, freq='D')


In [18]:
combined_data = pd.concat([past_data, future_data], ignore_index=False)
combined_data

,Dates,Prices
0,2020-10-31,10.100000
1,2020-11-01,10.106667
2,2020-11-02,10.113333
3,2020-11-03,10.120000
4,2020-11-04,10.126667
...,...,...
2030-03-09 00:00:00,2030-12-08,12.541361
2030-03-10 00:00:00,2030-12-09,12.541361
2030-03-11 00:00:00,2030-12-10,12.541361
2030-03-12 00:00:00,2030-12-11,12.541361


In [19]:
data = combined_data[combined_data['Dates'] == date_dt]
if not data.empty:
	print(data['Prices'].values[0])
else:
	print(f"No data found for date {date_dt}")
 


12.541360817968076


In [20]:
11.029695171134112


11.029695171134112

In [21]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=combined_data['Dates'], y=combined_data['Prices'], mode='lines', name='Past natural gas prices'))
#fig.add_trace(go.Scatter(x=future_data['Dates'], y=future_data['Prices'], mode='lines', name='Future natural gas prices'))
fig.update_layout(title='Past and Future Natural Gas Prices',
                  xaxis_title='Date',
                  yaxis_title='Price')
fig.show()

In [22]:
best_model_info



{'class': statsmodels.tsa.holtwinters.model.ExponentialSmoothing,
 'mape': 0.0079300348713607,
 'params': {'trend': 'add',
  'damped_trend': True,
  'seasonal': 'mul',
  'seasonal_periods': 365,
  'initialization_method': 'legacy-heuristic',
  'use_boxcox': False},
 'r2': 0.935679622644676}

In [23]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Suppose your report entry looks like this:
best_entry = model_report["Exponential Smoothing_49"]

best_model_class = best_entry['class']     # this is the class, not fitted
best_params = best_entry['params']

# Instantiate with your training series (let’s call it train_series)
model = best_model_class(train_arr, **best_params)

# Now fit it
fitted_model = model.fit()
print(fitted_model.summary())



                       ExponentialSmoothing Model Results                       
Dep. Variable:                   Prices   No. Observations:                 1157
Model:             ExponentialSmoothing   SSE                              0.015
Optimized:                         True   AIC                         -12266.443
Trend:                   Multiplicative   BIC                         -10396.616
Seasonal:                      Additive   AICC                        -11912.474
Seasonal Periods:                   365   Date:                 Sun, 17 Aug 2025
Box-Cox:                          False   Time:                         16:00:00
Box-Cox Coeff.:                    None                                         
                           coeff                 code              optimized      
----------------------------------------------------------------------------------
smoothing_level                0.9674677                alpha                 True
smoothing_trend       

In [24]:
model = ExponentialSmoothing(
    train_arr,
    trend="add",
    seasonal="add",
    seasonal_periods=365,
    initialization_method="estimated"
).fit()
print(model.summary())


                       ExponentialSmoothing Model Results                       
Dep. Variable:                   Prices   No. Observations:                 1157
Model:             ExponentialSmoothing   SSE                              0.008
Optimized:                         True   AIC                         -13048.311
Trend:                         Additive   BIC                         -11183.538
Seasonal:                      Additive   AICC                        -12696.688
Seasonal Periods:                   365   Date:                 Sun, 17 Aug 2025
Box-Cox:                          False   Time:                         16:00:02
Box-Cox Coeff.:                    None                                         
                           coeff                 code              optimized      
----------------------------------------------------------------------------------
smoothing_level                1.0000000                alpha                 True
smoothing_trend       

/Users/olasuaifan/opt/anaconda3/envs/quant/lib/python3.11/site-packages/statsmodels/tsa/holtwinters/model.py:903: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



In [25]:
d=pd.read_csv('artifacts/data.csv')

In [26]:
d

,Dates,Prices
0,10/31/20,10.10
1,11/30/20,10.30
2,12/31/20,11.00
3,1/31/21,10.90
4,2/28/21,10.90
5,3/31/21,10.90
6,4/30/21,10.40
7,5/31/21,9.84
8,6/30/21,10.00
9,7/31/21,10.10


In [ ]:
#Natural gas contract price

value=selling_price-purchase_price-cost
selling_price=sell_price*gas_volume
purchase_price_cost=gas_volume*purchase_price

cost=storage_cost +injection_withdrawal_cost+transportation_cost
storage_cost=storage_fees* storage_months
injection_withdrawal_cost= injection_withdrawal_fees*injection_withrdawl_times
gas_volum<max_storage_volume 



In [ ]:
#Natural gas contract price

value=selling_price-purchase_price-cost
selling_price=sell_price*gas_volume
purchase_price_cost=gas_volume*purchase_price

cost=storage_cost +injection_withdrawal_cost+transportation_cost
storage_cost=storage_fees* storage_months
injection_withdrawal_cost= injection_withdrawal_fees*injection_withrdawl_times
gas_volum<max_storage_volume 
